In [1]:
from mnist_utils import load_mnist
import ml.array as ml

# collect data
train_images, train_labels, test_images, test_labels = load_mnist()

mnist_train.csv has been saved.
mnist_test.csv has been saved.
DataFrames loaded with MNIST data.
Arrays loaded with MNIST data.


In [55]:
# one-hot encoding of train and test labels
def one_hot_encode(data, classes):
    out = []
    for d in data:
        one_hot = [0 for _ in range(classes)]
        one_hot[int(d)] = 1
        out.append(one_hot)
    return out

train_labels_onehot = ml.Array(one_hot_encode(train_labels.eval(), 10))
test_labels_onehot = ml.Array(one_hot_encode(test_labels.eval(), 10))

In [164]:
batch_size = 32
x = ml.Array(train_images.eval()[:batch_size].tolist())
y = ml.Array(train_labels_onehot.eval()[:batch_size].tolist())

In [313]:
# create the model and computational graphs
def softmax(x):
    # exps = x.exp()
    # return exps / exps.sum(1, keepdims=True)
    return x / x.sum(1, keepdims=True)

class Linear:

    def __init__(self, inp_shape, out_shape):
        self.w = ml.rand([inp_shape,out_shape])
        self.b = ml.rand([out_shape])

    def __call__(self, x):
        return x @ self.w + self.b.unsqueeze(0)

l1 = Linear(784, 10)

params = [l1.w, l1.b]

pred = softmax(l1(x).relu())
loss = -(y * pred.log()).sum()
loss.build_backward()

In [314]:
loss

array(73.49131, dtype=float32)

In [322]:
class SGD:
    def __init__(self, lr, params):
        self.lr = lr
        self.params = params
        self.temp_arrays = [loss.node.backend.empty(p.node.shape) for p in params]

    def step(self):
        for i, p in enumerate(params):
            t = self.temp_arrays[i]
            p.grad().set_reeval()
            loss.node.backend.multiply(self.lr, p.grad().eval(), t)
            loss.node.backend.negative(t, t)
            loss.node.backend.add(p.node.data, t, p.node.data)

class Adam:
    def __init__(self, lr, params, beta1=0.9, beta2=0.999, epsilon=1e-8, backend=loss.node.backend):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.params = params
        self.t = 0  # Time step
        self.backend = backend  # The NumpyBackend instance

        # Initialize moment vectors
        self.m = [self.backend.empty(p.node.shape) for p in params]
        self.v = [self.backend.empty(p.node.shape) for p in params]

        # Initialize moments to zero
        for i in range(len(self.params)):
            self.backend.fill_inplace(self.m[i], 0)
            self.backend.fill_inplace(self.v[i], 0)

    def step(self):
        self.t += 1
        for i, p in enumerate(self.params):
            grad = p.grad().eval()
            
            # Update biased first moment estimate
            self.backend.multiply(self.beta1, self.m[i], self.m[i])  # in-place scale by beta1
            self.backend.multiply((1 - self.beta1), grad, grad)  # scale grad by (1-beta1)
            self.backend.add(self.m[i], grad, self.m[i])  # in-place update of first moment

            # Update biased second moment estimate
            self.backend.multiply(self.beta2, self.v[i], self.v[i])  # in-place scale by beta2
            self.backend.square(grad, grad)  # square the gradients in-place
            self.backend.multiply((1 - self.beta2), grad, grad)  # scale squared grad by (1-beta2)
            self.backend.add(self.v[i], grad, self.v[i])  # in-place update of second moment

            # Compute bias-corrected first moment estimate
            m_hat = self.m[i] / (1 - self.beta1 ** self.t)
            # Compute bias-corrected second moment estimate
            v_hat = self.v[i] / (1 - self.beta2 ** self.t)

            # Update parameters
            self.backend.sqrt(v_hat, v_hat)  # in-place square root
            self.backend.add(v_hat, self.epsilon, v_hat)  # in-place addition of epsilon
            self.backend.reciprocal(v_hat, v_hat)  # in-place reciprocal
            self.backend.multiply(m_hat, v_hat, v_hat)  # in-place multiplication with m_hat
            self.backend.multiply(self.lr, v_hat, v_hat)  # in-place multiplication with learning rate
            self.backend.add(p.node.data, -v_hat, p.node.data)  # update parameters


In [328]:
lr = 0.1
iter = 5000
reports = 5

# optim = SGD(lr, params)
optim = Adam(lr, params)

for it in range(iter):  
    if (it % (iter//reports) == 0):
        print("\tloss = ", loss)
    loss.eval()
    loss.zero_grad()
    loss.set_reeval()
    optim.step()

	loss =  array(75.9725, dtype=float32)
	loss =  array(75.9725, dtype=float32)
	loss =  array(75.9725, dtype=float32)
	loss =  array(75.9725, dtype=float32)
	loss =  array(75.9725, dtype=float32)
